In [1]:
import pandas as pd
from tqdm.auto import tqdm
import warnings
from dev import *
from dev.aggregate import clear_aggregated_cache
from concurrent.futures import ProcessPoolExecutor, as_completed

warnings.filterwarnings("ignore")
clear_aggregated_cache()


# Worker function
def worker(eval_setup, dataset_name, source_name, attack_name, attack_strength):
    performance = get_performance(
        dataset_name,
        source_name,
        attack_name,
        attack_strength,
        mode=eval_setup,
    )
    quality = get_quality(
        dataset_name,
        source_name,
        attack_name,
        attack_strength,
        mode=eval_setup,
    )
    record = {
        "eval_setup": eval_setup,
        "dataset_name": dataset_name,
        "source_name": source_name,
        "attack_name": attack_name,
        "attack_strength": attack_strength,
    }
    record.update({k: v for k, v in performance.items()})
    record.update({k: v for k, v in quality.items()})
    return record


# Configuration for ThreadPoolExecutor
max_workers = 16  # Set the maximum number of workers
job_limit = float("inf")  # Set the job limit, can be set to float('inf') for no limit

# Prepare data for job submission
json_dict = get_all_json_paths(
    lambda _dataset_name, _attack_name, _attack_strength, _source_name, _result_type: (
        (_attack_name is not None)
        and (_attack_strength is not None)
        and (not _source_name.startswith("real"))
    )
)
record_keys = sorted(
    list(set([(key[0], key[3], key[1], key[2]) for key in json_dict.keys()])),
    key=lambda x: tuple((item is None, item) for item in x),
)
total = min(len(EVALUATION_SETUPS) * len(record_keys), job_limit)

# Parallel execution
records = []
with ProcessPoolExecutor(max_workers=max_workers) as executor:
    futures = []
    for eval_setup in EVALUATION_SETUPS.keys():
        for dataset_name, source_name, attack_name, attack_strength in record_keys:
            if len(futures) < job_limit:
                future = executor.submit(
                    worker,
                    eval_setup,
                    dataset_name,
                    source_name,
                    attack_name,
                    attack_strength,
                )
                futures.append(future)
            else:
                break

    # Collect results
    for future in tqdm(as_completed(futures), total=total):
        records.append(future.result())

# Convert to DataFrame and save
table = pd.DataFrame(records)
table.to_pickle("all_result_v2.pkl")

  0%|          | 0/3510 [00:00<?, ?it/s]

In [2]:
pd.read_pickle("all_result_v2.pkl")

,eval_setup,dataset_name,source_name,attack_name,attack_strength,acc_1,auc_1,low100_1,low1000_1,legacy_fid,clip_fid,psnr,ssim,nmi,lpips,watson,aesthetics,artifacts,clip_score
0,removal,dalle3,stable_sig,2x_regen,100.0,0.5087,0.488747,0.0054,0.0012,"(11.950457663492841, 0.0)","(1.3606403138919916, 0.0)","(20.47542227178457, 2.980594847827085)","(0.5594954632925753, 0.12416238932677727)","(1.1376805543615127, 0.048204044191881276)","(0.22591577281802894, 0.06512837279476806)","(6317276.83846875, 100216072.26131058)","(0.18944756713509558, 0.48381525164482214)","(-0.07942363864183426, 0.2479116364173105)","(-0.0033872624933719633, 0.01145103544505203)"
1,removal,dalle3,stable_sig,4x_regen,20.0,0.5042,0.481144,0.0066,0.0008,"(8.909160145983718, 1.7763568394002505e-15)","(1.0928339725712843, 2.220446049250313e-16)","(20.508247034351115, 2.8358443815984233)","(0.5867356570187698, 0.11778348324035973)","(1.146848819034788, 0.0498435155977178)","(0.22683387656509876, 0.07120185891203544)","(4138151.75265625, 25589490.193882782)","(0.41122494542971255, 0.4798492185768808)","(-0.11326520991325378, 0.25697312730366034)","(-0.005093095508404076, 0.011379850277192748)"
2,removal,dalle3,stable_sig,4x_regen,30.0,0.5012,0.449200,0.0024,0.0002,"(17.888071042748663, 0.0)","(2.7500248775454286, 4.440892098500626e-16)","(19.65400689640594, 2.705346509194197)","(0.5344035428065541, 0.11545415436986041)","(1.127824553214518, 0.043433599891751405)","(0.3008461918607354, 0.08463793766497513)","(8931907.13915625, 143231177.30480567)","(0.5452098059877754, 0.576450321878149)","(-0.20840495532751083, 0.2989573284503646)","(-0.009817458860576153, 0.01455967712907178)"
3,removal,dalle3,stable_sig,2x_regen,20.0,0.5298,0.537489,0.0098,0.0018,"(4.808336591091461, 0.0)","(0.38104822151503015, 0.0)","(23.249902572912536, 3.1788729755903278)","(0.6833927196897406, 0.10811329418038841)","(1.187487968029458, 0.059042823517310404)","(0.12361973528191447, 0.04310015426425451)","(3685886.7109375, 35441197.346991636)","(0.0821058354973793, 0.3396569703025583)","(0.015196512311697006, 0.1788275712807598)","(-0.0010460978876799345, 0.008120075928693423)"
4,removal,dalle3,stable_sig,2x_regen,80.0,0.5087,0.492008,0.0066,0.0004,"(11.85307269061565, 1.7763568394002505e-15)","(1.3150684091931453, 0.0)","(20.932713699145907, 3.0133695906542903)","(0.5799233074526616, 0.1212747009504219)","(1.1444338591543795, 0.04932926812417305)","(0.20959734423458576, 0.06101417977832503)","(12059335.25778125, 294961828.92049134)","(0.15415528405457735, 0.4727175051389172)","(-0.05846917560696602, 0.24483961455761155)","(-0.0036702554803341625, 0.01119221773768658)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3505,combined,mscoco,tree_ring,regen_vae,1.0,0.9701,0.995110,0.9158,0.6734,"(32.30219006313786, 0.0)","(8.553331940918596, 1.2560739669470201e-15)","(27.263753653898537, 2.683190538010998)","(0.777754659706848, 0.08628614721929016)","(1.251506754616142, 0.05224554887225737)","(0.2908310448694974, 0.10998827119360984)","(1006748.2806015625, 1036094.0235993011)","(1.3718504326343535, 0.5830013178231387)","(-0.7156772080212832, 0.3360481485055953)","(-0.01262418560788501, 0.013384532496308063)"
3506,combined,mscoco,tree_ring,regen_diffusion_prompt,40.0,0.9909,0.999460,0.9908,0.9744,"(8.519496329245271, 1.2560739669470201e-15)","(1.888302238615978, 0.0)","(24.05654631040253, 3.0266589588224835)","(0.7141699862890326, 0.10833890398767278)","(1.2020547554344245, 0.05453609033473679)","(0.12138553906138987, 0.03244967139465769)","(1082973.458832031, 823102.8057725636)","(0.3806292562484741, 0.37953994081225273)","(-0.22523674069344995, 0.22868993247301186)","(-0.0009867625538026914, 0.009243261804851114)"
3507,combined,mscoco,tree_ring,regen_vae,3.0,0.9934,0.999735,0.9946,0.9772,"(15.62090945216815, 1.2560739669470201e-15)","(4.0706652402048755, 6.473657049138938e-16)","(30.088003675124234, 2.804429882117367)","(0.8595952869024828, 0.059349356837921786)","